In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

from subprocess import check_call, check_output
from pathlib import Path
import json
import sys
import re
import pandas as pd
import numpy as np

In [ ]:
BASE_PATH = Path("Set before using")

LEFT = BASE_PATH / "6-hd-left"
RIGHT = BASE_PATH / "6-hd-right"
STRAIGHT = BASE_PATH / "6-hd-straight"

TOTAL_BITRATE = 6000

In [ ]:
def call(command) -> int:
    """Call a subprocess in the shell and return code"""
    return check_call(command, shell=True)


def call_output(command) -> str:
    """Call a subprocess in the shell and return output"""
    return check_output(command, shell=True).decode(sys.stdout.encoding).strip()


def parse_metric_score(content: str, metric: str) -> float:
    """Parse the metric score from console output"""
    metric_patterns = {"ssim": r"All:(\d*\.\d*)", "psnr": r"average:(\d*\.\d*)"}
    score = re.search(metric_patterns[metric], content, re.DOTALL)
    return float(score.group(1)) if score else 0.0


def process(file: Path, bitrate: float, key: str, prio: float):
    mp4_file = file.with_suffix(".mp4")
    call(
        f"gst-launch-1.0 filesrc location={file} ! videoparse width=1920 height=1080 framerate=20/1 format=2 ! autovideoconvert ! "
        f"x264enc tune=zerolatency speed-preset=ultrafast bitrate={int(bitrate)} sliced-threads=true byte-stream=true threads=1 key-int-max=15 intra-refresh=true ! "
        f"mp4mux ! filesink location={mp4_file}"
    )
    output = call_output(
        "ffmpeg -hide_banner"
        f" -s:v 1920x1080 -r 20 -pix_fmt yuv420p"
        f" -i {file} -i {mp4_file}"
        f' -lavfi "[0:v]scale=1920x1080[main];[main][1:v]ssim;'
        f'[0:v]scale=1920x1080[main];[main][1:v]psnr" -f null - 2>&1'
    )
    distorted_yuv = file.with_name(file.stem + "_dist" + file.suffix)
    call(f"ffmpeg -y -i {mp4_file} {distorted_yuv}")
    vmaf_json = call_output(
        f"docker run --rm -v {file.parent}:/files vmaf:latest yuv420p 1920 1080"
        f" /files/{file.name} /files/{distorted_yuv.name} --out-fmt json"
    )
    call(f"rm {mp4_file} {distorted_yuv}")
    return {
        "key": key,
        "prio": prio,
        "psnr": parse_metric_score(output, "psnr"),
        "ssim": parse_metric_score(output, "ssim"),
        "vmaf": json.loads(vmaf_json)["aggregate"]["VMAF_score"],
    }

In [ ]:
df_uniform = pd.DataFrame(
    [
        process(RIGHT / "front_1080p.yuv", TOTAL_BITRATE / 6, "uniform-right-f", 9.25),
        process(RIGHT / "front_left_1080p.yuv", TOTAL_BITRATE / 6, "uniform-right-fl", 5.75),
        process(RIGHT / "front_right_1080p.yuv", TOTAL_BITRATE / 6, "uniform-right-fr", 9.5),
        process(RIGHT / "rear_1080p.yuv", TOTAL_BITRATE / 6, "uniform-right-r", 4.75),
        process(RIGHT / "rear_left_1080p.yuv", TOTAL_BITRATE / 6, "uniform-right-rl", 4.63),
        process(RIGHT / "rear_right_1080p.yuv", TOTAL_BITRATE / 6, "uniform-right-rr", 5.38),
        process(LEFT / "front_1080p.yuv", TOTAL_BITRATE / 6, "uniform-left-f", 9.25),
        process(LEFT / "front_left_1080p.yuv", TOTAL_BITRATE / 6, "uniform-left-fl", 9.38),
        process(LEFT / "front_right_1080p.yuv", TOTAL_BITRATE / 6, "uniform-left-fr", 6.13),
        process(LEFT / "rear_1080p.yuv", TOTAL_BITRATE / 6, "uniform-left-r", 4.75),
        process(LEFT / "rear_left_1080p.yuv", TOTAL_BITRATE / 6, "uniform-left-rl", 5.38),
        process(LEFT / "rear_right_1080p.yuv", TOTAL_BITRATE / 6, "uniform-left-rr", 4.88),
        process(STRAIGHT / "front_1080p.yuv", TOTAL_BITRATE / 6, "uniform-straight-f", 10.0),
        process(STRAIGHT / "front_left_1080p.yuv", TOTAL_BITRATE / 6, "uniform-straight-fl", 6.88),
        process(STRAIGHT / "front_right_1080p.yuv", TOTAL_BITRATE / 6, "uniform-straight-fr", 7.50),
        process(STRAIGHT / "rear_1080p.yuv", TOTAL_BITRATE / 6, "uniform-straight-r", 5.00),
        process(STRAIGHT / "rear_left_1080p.yuv", TOTAL_BITRATE / 6, "uniform-straight-rl", 4.88),
        process(STRAIGHT / "rear_right_1080p.yuv", TOTAL_BITRATE / 6, "uniform-straight-rr", 4.38),
    ]
)

In [ ]:
df_prio = pd.DataFrame(
    [
        process(RIGHT / "front_1080p.yuv", TOTAL_BITRATE * 0.31, "prio-right-f", 9.25),
        process(RIGHT / "front_left_1080p.yuv", TOTAL_BITRATE * 0.155, "prio-right-fl", 5.75),
        process(RIGHT / "front_right_1080p.yuv", TOTAL_BITRATE * 0.31, "prio-right-fr", 9.5),
        process(RIGHT / "rear_roi_1080p.yuv", TOTAL_BITRATE * 0.075, "prio-right-r", 4.75),
        process(RIGHT / "rear_left_roi_1080p.yuv", TOTAL_BITRATE * 0.075, "prio-right-rl", 4.63),
        process(RIGHT / "rear_right_roi_1080p.yuv", TOTAL_BITRATE * 0.075, "prio-right-rr", 5.38),
        process(LEFT / "front_1080p.yuv", TOTAL_BITRATE * 0.31, "prio-left-f", 9.25),
        process(LEFT / "front_left_1080p.yuv", TOTAL_BITRATE * 0.31, "prio-left-fl", 9.38),
        process(LEFT / "front_right_1080p.yuv", TOTAL_BITRATE * 0.155, "prio-left-fr", 6.13),
        process(LEFT / "rear_roi_1080p.yuv", TOTAL_BITRATE * 0.075, "prio-left-r", 4.75),
        process(LEFT / "rear_left_roi_1080p.yuv", TOTAL_BITRATE * 0.075, "prio-left-rl", 5.38),
        process(LEFT / "rear_right_roi_1080p.yuv", TOTAL_BITRATE * 0.075, "prio-left-rr", 4.88),
        process(STRAIGHT / "front_1080p.yuv", TOTAL_BITRATE * 0.31, "prio-straight-f", 10.0),
        process(STRAIGHT / "front_left_1080p.yuv", TOTAL_BITRATE * 0.31, "prio-straight-fl", 6.88),
        process(STRAIGHT / "front_right_1080p.yuv", TOTAL_BITRATE * 0.155, "prio-straight-fr", 7.50),
        process(STRAIGHT / "rear_roi_1080p.yuv", TOTAL_BITRATE * 0.075, "prio-straight-r", 5.00),
        process(STRAIGHT / "rear_left_roi_1080p.yuv", TOTAL_BITRATE * 0.075, "prio-straight-rl", 4.88),
        process(STRAIGHT / "rear_right_roi_1080p.yuv", TOTAL_BITRATE * 0.075, "prio-straight-rr", 4.38),
    ]
)

In [ ]:
df_uniform.to_csv(BASE_PATH / "uniform.csv")
df_prio.to_csv(BASE_PATH / "prio.csv")

In [ ]:
df_uniform = pd.read_csv(BASE_PATH / "uniform.csv")
df_prio = pd.read_csv(BASE_PATH / "prio.csv")

In [ ]:
uniform_vmaf = np.mean(df_uniform.prio * df_uniform.vmaf * 0.1)
uniform_psnr = np.mean(df_uniform.prio * df_uniform.psnr * 0.1)

prio_vmaf = np.mean(df_prio.prio * df_prio.vmaf * 0.1)
prio_psnr = np.mean(df_prio.prio * df_prio.psnr * 0.1)